<a href="https://colab.research.google.com/github/EmreAPAYDIN/phyton/blob/master/Tensor_and_Gradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.10.1 Tensor and Gradients

The tensor is the most fundamental computation unit in TensorFlow and it is used  to represent outputs of an operation. A tensor can be created by operations such  as **tf.constant**, **tf.matmul**, etc. Tensor does not store the values of the  operation’s outputs but provides access to the computation of those values in a  TensorFlow session. In TensorFlow 2.0, there is no need to run a session manually,  as in eager execution, graphs and sessions are designed to stay in the backend. For  examples, in the matrix multiplication as shown below, matrices can be created by  **tf.constant** and the multiplication is computed by **tf.matmul** whose output  is another matrix.

In [ ]:
# Matrix multiplication in TensorFlow by Tensor
import tensorflow as tf
a = tf.constant([[1, 2], [1, 2]])
# tf.Tensor(
# [[1 2]
# [1 2]], shape=(2, 2), dtype=int32)
b = tf.constant([[1], [2]])
# tf.Tensor(
# [[1]
# [2]], shape=(2, 1), dtype=int32)
c = tf.matmul(a, b)
# tf.Tensor(
# [[5]
# [5]], shape=(2, 1), dtype=int32)
print(c)

In the forward propagation of deep neural networks, the tensors are automatically connected by each other as a graph. Based on the graph and the automaticdifferentiation technique provided by TensorFlow, gradients can be computed in  the back-propagation. TensorFlow 2.0 provides tf.GradientTape to compute  gradients of recorded operations with respect to its input variables. For example,  the code below shows an example of computing gradients in back-propagation.  The forward propagation and the computation of loss are within the scope of  tf.GradientTape, while the back-propagation and the update of weights  are outside the scope. The tf.GradientTape records all operations that are  executed within the scope onto a tape. Then the gradients associated with each  recorded operation and its input variables are computed by reverse-mode automatic  differentiation. Once the function tape.gradient() is called, the resources  held by tf.GradientTape are released.

In [ ]:
!pip install tensorlayer

In [6]:
#Gradients computation in TensorFlow and TensorLayer.

import tensorflow as tf
import tensorlayer as tl
def train(model, dataset, optimizer):
# given a model which is an instance of Model by TensorLayer
# traverse the dataset where x is input and y is target output
  for x, y in dataset:
    # create the scope of gradient tape
    with tf.GradientTape() as tape:
      prediction = model(x) # forward propagation
      loss = loss_fn(prediction, y) # loss function
    # back-propagation and computing gradients, then the resources held by the GradientTape are released
    gradients = tape.gradient(loss, model.trainable_weights)
    # apply the gradients to weights and update the weights by the optimizer
    optimizer.apply_gradients(zip(gradients,model.trainable_weights))

# 1.10.2 Define a Model

In TensorLayer 2.0, Model is an entity that consists of multiple Layers and defines  the propagation between the Layers. TensorLayer 2.0 provides two sets of APIs  to define a model. Static model APIs allow users to build a model fluently and  dynamic model APIs provide more flexibility in the forward propagation. A static  model requires users to manually construct a graph and compile it. Once the model  is compiled, the forward propagation cannot be changed. Unlike the static model, the  dynamic model can be executed eagerly like Python normally does and the forward  propagation is mutable.

In the implementation of models, as shown in the examples below, the difference between a static model and a dynamic model can be summarized in two aspects. First, when layers in a static model are declared, the connection between layers (i.e., the forward propagation) is defined explicitly at the same time. Based on the  connection, for each layer, TensorLayer can automatically infer the size of input  variables from previous layers and then construct weights. When the Model is  finally instanced, only inputs and outputs need to be specified and TensorLayer  automatically builds a graph based on the connection. However, when a dynamic  model is initialized, the forward propagation is still unknown until it is defined in the  function forward later. Thus, the size of input variables cannot be automatically  inferred and it has to been manually provided via the argument in_channels.

Second, the forward propagation of a static model is fixed once the model is constructed, so it is easier to accelerate the computation of a static model. TensorFlow  2.0 provides a new feature called tf.function which can be used as a decorator  and accelerate the computation. Unlike the static model, the forward propagation  in a dynamic model can be more flexible. For example, the forward flow can be  controlled based on input values or arguments specified by users. Users are also  allowed to use or abandon any layer in the forward propagation of a dynamic model.

In [ ]:
!pip install --upgrade tensorlayer[all]

In [ ]:
# An example of a static model: multilayer perceptron (MLP)
import tensorflow as tf
from tensorlayer.layers import Input, Dense
from tensorlayer.models import Model
# a multilayer perceptron (MLP) model with three dense layers
def get_mlp_model(inputs_shape):
  ni = Input(inputs_shape)
  # since the connection between layers is explicitly defined
  # in_channels of each layer is automatically inferred
  nn = Dense(n_units=800, act=tf.nn.relu)(ni)
  nn = Dense(n_units=800, act=tf.nn.relu)(nn)
  nn = Dense(n_units=10, act=tf.nn.relu)(nn)
  # automatic build a model based on the connection between layers
  M = Model(inputs=ni,outputs=nn)
  return M
MLP = get_mlp_model([None, 784])
# switch to evaluation mode
MLP.eval()
# ingest data into the model
# the computation can be accelerated by using @tf.function in TensorFlow 2.0
outputs = MLP(data)